In [1]:
#pip install pandas openpyxl

In [2]:
#pip install python-docx

In [3]:
#!pip install faker

In [4]:
import os
import PyPDF2
import pandas as pd
from PIL import Image
import re
from PyPDF2 import PdfReader
from faker import Faker
import pandas as pd
from docx import Document

In [5]:
name_regex = re.compile(r'(?P<name>[A-Za-z]+ [A-Za-z]+)')
phone_regex = re.compile(r'(?P<phone>\+?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9})')
email_regex = re.compile(r'(?P<email>[\w\.-]+@[\w\.-]+)')

In [6]:
def generate_dummy_data(num_records):
    fake = Faker()
    for _ in range(num_records):
        name = fake.name()
        phone = fake.phone_number()
        email = fake.email()
    return name, phone, email

In [7]:
def extract_info_from_resume(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        content = ''
        d_name, d_phone, d_email = generate_dummy_data(1)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            content += page.extract_text()

        name_match = name_regex.search(content)
        if name_match:
            name = name_match.group('name')
            content = content.replace(name_match.group('name'), d_name)
        
        phone_match = phone_regex.search(content)
        if phone_match:
            phone = phone_match.group('phone')
            content = content.replace(phone_match.group('phone'), d_phone)
        
        email_match = email_regex.search(content)
        if email_match:
            email = email_match.group('email')
            content = content.replace(email_match.group('email'), d_email)
        
 
        return name, phone, email,content,d_name, d_phone, d_email
        
    

In [8]:
def main(resume_folder):
    data = []

    
    for filename in os.listdir('giekel'):
        if filename.endswith('.pdf'):
            file_path = os.path.join('giekel', filename)
            name, phone, email,content,d_name, d_phone, d_email = extract_info_from_resume(file_path)
            data.append({'Name': name, 'Phone': phone, 'Email': email,'dummy_Name': d_name, 'dummy_Phone': d_phone, 'dummy_Email': d_email})
    df = pd.DataFrame(data)
    
    return content,df

if __name__ == "__main__":
    resume_folder = 'giekel'
    output_excel = "parsed_resumes2.xlsx"
    x,df=main(resume_folder)
    df.to_excel(output_excel, index=False, engine='openpyxl', encoding='latin1')
    doc = Document()
    doc.add_paragraph(x)
    doc.save('sample.docx')
    